<h1 align="center">BSc Thesis</h1>

<h2 align="center">Parkinson’s disease and the Medial Geniculate Nucleus: <br/> A 7-Tesla MRI comparison of structural volume between patients and healthy controls</h2>


In [1]:
%load_ext rpy2.ipython

# python packages
import os
import sys
import numpy as np
import pandas as pd
from scipy.stats import mannwhitneyu

# to run R code in python
import rpy2.robjects.numpy2ri
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector

# my python packages
sys.path.insert(1, 'src')
from visualizations import boxplots, scatterplots
from dataProcessing import getFinalData
sys.path.insert(1, 'src/BayesianMannWhitneyTest')
from truncation import *
from gibbs import *
from BayesFactorWilcoxon import *

C:\Users\odyse\anaconda3\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [3]:
def BayesFactor(data):
    volHC = data.loc[data['group'] == "HC"]["meanVol"]
    volPD = data.loc[data['group'] == "PD"]["meanVol"]
    # ignore warning
    np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
    result = rankSumGibbsSampler(volHC, volPD, nBurnin=1)
    rHat = result[0]
    deltaSamplesMatrixFlat = result[2]
    # pass variable to R
    %R -i deltaSamplesMatrixFlat
    # define R variable
    %R -o vec vec = as.vector(deltaSamplesMatrixFlat)
    rpy2.robjects.numpy2ri.activate()
    # one-sided hypothesis: healthy controls will have a greater mean than pd patients
    bf = computeBayesFactorWilcoxon(vec, 1/sqrt(2), "right")[0]
    U, p = mannwhitneyu(volHC, volPD, method="exact")
    
    median, low, high = posteriorCRI(onesided="right", deltaSamples=deltaSamplesMatrixFlat, criVal=0.95)
    
    n = len(volHC.index)
    posteriorMedian = round(median, 2),
    CRI = [round(low, 2), round(high, 2)]
    meanHC = np.mean(volHC)
    meanPD = np.mean(volPD)
    
    out = [n, bf, median, CRI, meanHC, meanPD]
    
    # an rHat of ~ 1 shows that there was convergence, which is what we want
    print("\nrHat =", rHat)
    print("BF =", bf)
    print("U statistic =", U, "\n")
    
    return out

In [4]:
def simulatedBayesFactors(simSampleSizes):
    results = []
    print("Be patient, the simulation takes several minutes :) \n")
    for i in simSampleSizes:
        simData = pd.read_csv("outputData/simulatedData" + i + ".csv")
        print("Now computing BF for simulated data with n = " + i)
        results.append(BayesFactor(simData))
    
    simBFs = pd.DataFrame(data=results, columns=["n", "BF", "posteriorMedian", "95% CRI", "meanHC", "meanPD"])
        
    return simBFs

In [9]:
def main():
    data = getFinalData()
    # display(data.head())
    # boxplots(data)
    # scatterplots(data)
    result = BayesFactor(data)
    simSampleSizes = ["11", "15", "20", "30", "50", "75"]
    simBFs = simulatedBayesFactors(simSampleSizes)
    
    return result, simBFs

result, simBFs = main()

The mean volume of the left hemisphere is 122.80682145454544 and the mean volume of the right hemisphere is 120.76136727272727 
This difference between the hemispheric volumes of the mgn is not significant for the healthy controls, W = 30.0 and p = 0.8310546875 for a two-sided alternative hypothesis.

The mean volume of the left hemisphere is 99.9886359090909 and the mean volume of the right hemisphere is 105.94318227272726 
This difference between the hemispheric volumes of the mgn is not significant for the PD patients, W = 27.0 and p = 0.8310546875 for a two-sided alternative hypothesis.

Starting chain:  1 / 5
	Now at iteration:  1 / 1000
	Now at iteration:  501 / 1000
Starting chain:  2 / 5
	Now at iteration:  1 / 1000
	Now at iteration:  501 / 1000
Starting chain:  3 / 5
	Now at iteration:  1 / 1000
	Now at iteration:  501 / 1000
Starting chain:  4 / 5
	Now at iteration:  1 / 1000
	Now at iteration:  501 / 1000
Starting chain:  5 / 5
	Now at iteration:  1 / 1000
	Now at iteration

In [10]:
resultDF = pd.DataFrame(data=[result], columns=["n", "BF", "posteriorMedian", "95% CRI", "meanHC", "meanPD"])
display(resultDF)

,n,BF,posteriorMedian,95% CRI,meanHC,meanPD
0,11,1.004625,0.402225,"[0.03, 1.09]",121.784094,102.965909


In [11]:
display(simBFs)

,n,BF,posteriorMedian,95% CRI,meanHC,meanPD
0,11,0.471074,0.258702,"[0.02, 0.84]",116.602780,102.987514
1,15,2.128171,0.530443,"[0.05, 1.22]",126.692531,101.873059
2,20,1.867145,0.463695,"[0.04, 1.05]",120.721133,105.190203
3,30,15.840039,0.653092,"[0.18, 1.16]",127.065421,105.659567
4,50,24.171435,0.565525,"[0.17, 0.99]",119.101344,98.923458
5,75,33.312872,0.560860,"[0.22, 0.91]",120.275845,99.468072


In [13]:
if not os.path.exists("results"):
    os.mkdir("results")
resultDF.to_csv("results/result.csv")
simBFs.to_csv("results/simulatedBFs.csv")